## 高性能计算 query() eval()

In [1]:
import pandas as pd
import numpy as np

In [2]:
nrows,ncols = 100000,100
rng = np.random.RandomState(42)
df1,df2,df3,df4 = (pd.DataFrame(rng.rand(nrows,ncols)) for i in range(4))

In [3]:
%timeit df1+df2+df3+df4

59 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit pd.eval('df1+df2+df3+df4')

29.5 ms ± 760 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
#支持所有的算数运算符
result = df1+df2
result2 = pd.eval('df1+df2')

In [10]:
#比较运算符
result1 = (df1<df2) & (df2<=df3)
result2 = pd.eval('df1<df2<=df3')
np.allclose(result1,result2)

True

In [11]:
#位运算符
result1 = (df1<0.5) & (df2<0.5)
result2 = pd.eval('(df1<0.5) & (df2<0.5)')

In [12]:
#对象属性和索引
res = pd.eval('df2.T[0] + df3.iloc[1]')


#### 用df.eval()实现列间运算

In [13]:
df = pd.DataFrame(rng.rand(1000,3),columns=['a','b','c'])

In [16]:
res1 = pd.eval("(df.a+df.b)/(df.c-1)")
res2 = df.eval("(a+b)/(c-1)")
np.allclose(res1,res2)

True

In [19]:
#新建一个列
df.eval('d=(a+b)/c',inplace=True)

In [21]:
#使用局部变量
col_mean = df.mean(1)
res1 = df['a']+col_mean
res2 = df.eval('a+@col_mean')

#### 用df.query()实现查询

In [24]:
r1 = df.query('a<0.5 and b<0.5')
r2= df.query('a<0.5 and b<@col_mean')

# 层级索引


In [25]:
index = [('california',2000),('california',2010),('new york',2000),('new york',2010),('texas',2000),('texas',2010)]
populations = [123,21,321,4312,564,210]
pop = pd.Series(populations,index=index)

In [28]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['california', 'new york', 'texas'], [2000, 2010]],
           codes=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [29]:
pop = pop.reindex(index)

In [30]:
pop

california  2000     123
            2010      21
new york    2000     321
            2010    4312
texas       2000     564
            2010     210
dtype: int64

In [31]:
# 高维数据的多级索引 可以用一个带行列索引的简单df代替前面的多级索引
# unstack将带多级索引的series转换为普通索引的df
pop_df = pop.unstack()
pop_df

,2000,2010
california,123,21
new york,321,4312
texas,564,210


In [32]:
#stack实现相反的效果
pop_df.stack()

california  2000     123
            2010      21
new york    2000     321
            2010    4312
texas       2000     564
            2010     210
dtype: int64

In [33]:
#可以用series或者df表示更高维度的数据

#### 多级索引的创建方法

In [36]:
#将index参数设置为至少二维
df = pd.DataFrame(np.random.rand(4,2),index=[['a','c','a','d'],[1,2,3,4]])

In [37]:
df

,,0,1
a,1,0.781428,0.457865
c,2,0.412900,0.967536
a,3,0.017145,0.571639
d,4,0.253938,0.637255


In [39]:
#将元祖作为键的字典传递给pandas
pd.Series({('ca',123):81729})

ca  123    81729
dtype: int64

In [40]:
#显示的创建多维索引
pd.MultiIndex.from_arrays([['a','e','z'],[1,2,3]])

MultiIndex(levels=[['a', 'e', 'z'], [1, 2, 3]],
           codes=[[0, 1, 2], [0, 1, 2]])

In [41]:
#多维索引的等级名称
pop.index.names=['state','year']
pop

state       year
california  2000     123
            2010      21
new york    2000     321
            2010    4312
texas       2000     564
            2010     210
dtype: int64

In [42]:
#多级列索引

# 取值与切片

In [43]:
pop

state       year
california  2000     123
            2010      21
new york    2000     321
            2010    4312
texas       2000     564
            2010     210
dtype: int64

In [44]:
pop['california',2000]

123

In [45]:
pop['california']

year
2000    123
2010     21
dtype: int64

In [46]:
pop["california":'new york']

state       year
california  2000     123
            2010      21
new york    2000     321
            2010    4312
dtype: int64

In [47]:
pop[:,2000]

state
california    123
new york      321
texas         564
dtype: int64

# 多级索引行列转换

In [48]:
#有序的索引和无序的索引，如果不是有序的索引，那么切片操作会失败
index = pd.MultiIndex.from_product([['a','c','b'],[1,2]])
data = pd.Series(np.random.rand(6),index=index)
data.index.name =['char','int']
data

a  1    0.910892
   2    0.915873
c  1    0.472801
   2    0.205457
b  1    0.671244
   2    0.320408
dtype: float64

In [49]:
data = data.sort_index()

In [50]:
data

a  1    0.910892
   2    0.915873
b  1    0.671244
   2    0.320408
c  1    0.472801
   2    0.205457
dtype: float64

In [51]:
#stack unstack
pop

state       year
california  2000     123
            2010      21
new york    2000     321
            2010    4312
texas       2000     564
            2010     210
dtype: int64

In [52]:
pop.unstack(level=0)

state,california,new york,texas
year,,,
2000,123,321,564
2010,21,4312,210


In [53]:
pop.unstack(level=1)

year,2000,2010
state,,
california,123,21
new york,321,4312
texas,564,210


In [54]:
#索引的设置和重置 行列标签转换
pop_flat = pop.reset_index()

In [55]:
pop_flat

,state,year,0
0,california,2000,123
1,california,2010,21
2,new york,2000,321
3,new york,2010,4312
4,texas,2000,564
5,texas,2010,210


In [56]:
pop_flat.set_index(['state','year'])

0
state      year      
california 2000   123
           2010    21
new york   2000   321
           2010  4312
texas      2000   564
           2010   210

# 数据透视表

In [57]:
import seaborn as sns

In [58]:
titanic = sns.load_dataset('titanic')

In [59]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


In [62]:
#如果想看不同性别和不同等级仓的生还状态
titanic.groupby(['sex','class'])['survived'].mean().unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [63]:
titanic.pivot_table('survived',index='sex',columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [65]:
#多级数据透视表
age = pd.cut(titanic['age'],[0,18,80])
titanic.pivot_table('survived',['sex',age],'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

In [66]:
fare = pd.qcut(titanic['fare'],2)
titanic.pivot_table('survived',['sex',age],[fare,'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

In [67]:
titanic.pivot_table(index='sex',columns='class',aggfunc={'survived':sum,'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47